# **Pinecone tutorial**
This is the sample notebook from the Pinecone site.

# **How to run in Colab:**

This notebook can be run on Google Colab and stand alone python development environments. Click here to run on colab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/datariders/tutorials/blob/main/vectordb/pinecone/pinecone_tutorial.ipynb)


# **References:**
https://docs.pinecone.io/guides/get-started/quickstart


In [1]:
!pip3 install pinecone-client

In [2]:
import os
from getpass import getpass

#if os.environ.get("OPENAI_API_KEY") is None:
pinecone_api_key = getpass("🔑 Enter your Pinecone API key and hit Enter:")
os.environ["PINECONE_API_KEY"] = pinecone_api_key
print("Pinecone key has been entered")

🔑 Enter your Pinecone API key and hit Enter:··········
Pinecone key has been entered


In [3]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="957cfa5c-72fd-44f0-9a95-d74fe3bc2144")
print(" pc: ", pc)

 pc:  <pinecone.control.pinecone.Pinecone object at 0x7aa3e44b0100>


# **Create index**

Then create a serverless index named "quickstart" that performs nearest-neighbor search using the cosine distance metric for 2-dimensional vectors:

In [6]:
index_name = "quickstart"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=2,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

In [7]:
index = pc.Index(index_name)

index.upsert(
    vectors=[
        {"id": "vec1", "values": [1.0, 1.5]},
        {"id": "vec2", "values": [2.0, 1.0]},
        {"id": "vec3", "values": [0.1, 3.0]},
    ],
    namespace="ns1"
)

index.upsert(
    vectors=[
        {"id": "vec1", "values": [1.0, -2.5]},
        {"id": "vec2", "values": [3.0, -2.0]},
        {"id": "vec3", "values": [0.5, -1.5]},
    ],
    namespace="ns2"
)

{'upserted_count': 3}

In [8]:
print(index.describe_index_stats())

# Returns:
# {'dimension': 2,
#  'index_fullness': 0.0,
#  'namespaces': {'ns1': {'vector_count': 3}, 'ns2': {'vector_count': 3}},
#  'total_vector_count': 6}

{'dimension': 2,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 3}, 'ns2': {'vector_count': 3}},
 'total_vector_count': 6}


In [9]:
query_results1 = index.query(
    namespace="ns1",
    vector=[1.0, 1.5],
    top_k=3,
    include_values=True
)
print(" query_results1: ", query_results1)

query_results2 = index.query(
    namespace="ns2",
    vector=[1.0,-2.5],
    top_k=3,
    include_values=True
)
print(" query_results2: ", query_results2)

# Returns:
# {'matches': [{'id': 'vec1', 'score': 1.0, 'values': [1.0, 1.5]},
#              {'id': 'vec2', 'score': 0.868243158, 'values': [2.0, 1.0]},
#              {'id': 'vec3', 'score': 0.850068152, 'values': [0.1, 3.0]}],
#  'namespace': 'ns1',
#  'usage': {'read_units': 6}}
# {'matches': [{'id': 'vec1', 'score': 1.0, 'values': [1.0, -2.5]},
#              {'id': 'vec3', 'score': 0.998274386, 'values': [0.5, -1.5]},
#              {'id': 'vec2', 'score': 0.824041963, 'values': [3.0, -2.0]}],
#  'namespace': 'ns2',
#  'usage': {'read_units': 6}}

 query_results1:  {'matches': [{'id': 'vec1', 'score': 1.0, 'values': [1.0, 1.5]},
             {'id': 'vec2', 'score': 0.868243158, 'values': [2.0, 1.0]},
             {'id': 'vec3', 'score': 0.850068152, 'values': [0.1, 3.0]}],
 'namespace': 'ns1',
 'usage': {'read_units': 6}}
 query_results2:  {'matches': [{'id': 'vec1', 'score': 1.0, 'values': [1.0, -2.5]},
             {'id': 'vec3', 'score': 0.998274386, 'values': [0.5, -1.5]},
             {'id': 'vec2', 'score': 0.824041963, 'values': [3.0, -2.0]}],
 'namespace': 'ns2',
 'usage': {'read_units': 6}}


In [10]:
pc.delete_index(index_name)